# Menganalisis Risiko Gagal Bayar Peminjam



## Pendahuluan 
Setiap kali kita melakukan analisis, kita perlu merumuskan beberapa hipotesis yang perlu kita uji lebih lanjut. Terkadang, pengujian yang kita lakukan membawa kita untuk menerima hipotesis tersebut, adakalanya kita juga perlu menolaknya. Untuk membuat keputusan yang tepat dalam bisnis, kita harus memahami apakah asumsi yang kita buat sudah tepat atau belum.

Proyek kali ini bertujuan untuk menyiapkan laporan untuk divisi kredit suatu bank, dengan mencari tahu pengaruh status perkawinan seorang nasabah dan jumlah anak yang dimilikinya terhadap probabilitas gagal bayar dalam pelunasan pinjaman. Pihak bank sudah memiliki beberapa data mengenai kelayakan kredit nasabah.

Laporan ini akan dipertimbangkan pada saat membuat **penilaian kredit** untuk calon nasabah. **Penilaian kredit** digunakan untuk mengevaluasi kemampuan calon peminjam untuk melunasi pinjaman mereka.

### Tujuan: 
Menguji empat hipotesis:
1. Terdapat hubungan antara memiliki anak dan probabilitas seseorang melakukan gagal bayar pinjaman.
2. Terdapat hubungan antara status perkawinan dan probabilitas seseorang melakukan gagal bayar pinjaman.
3. Terdapat hubungan antara tingkat pendapatan dan probabilitas seseorang melakukan gagal bayar pinjaman.
4. Perbedaan tujuan pinjaman memengaruhi probabilitas seseorang melakukan gagal bayar pinjaman.

### Tahapan:
Data mengenai kelayakan kredit nasabah disimpan dalam *file* `/datasets/credit_scoring_eng.csv`. Tidak ada informasi terkait kualitas data tersebut, jadi perlu memeriksanya terlebih dahulu sebelum menguji hipotesis.

 
Proyek ini akan terdiri dari tiga tahap utama, yaitu:
1. Ikhtisar Data : Membaca file dan melakukan eksplorasi data
2. Pra-pemrosesan data : 
- Mengidentifikasi dan mengisi nilai-nilai yang hilang
- Mengganti tipe data bilangan riil dengan tipe integer
- Menghapus data duplikat
- Mengategorikan data
3. Pengujian Hipotesis 

## Buka *file* data dan baca informasi umumnya.

In [4]:
import pandas as pd 

In [5]:
try :
    df = pd.read_csv('credit_scoring_eng.csv')

except :
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Soal 1. Eksplorasi data

**Deskripsi Data**
- `children` - jumlah anak dalam keluarga
- `days_employed` - pengalaman kerja nasabah dalam hari
- `dob_years` - usia nasabah dalam tahun
- `education` - tingkat pendidikan nasabah
- `education_id` - pengidentifikasi untuk tingkat pendidikan nasabah
- `family_status` - pengidentifikasi untuk status perkawinan nasabah
- `family_status_id` - tanda pengenal status perkawinan
- `gender` - jenis kelamin nasabah
- `income_type` - jenis pekerjaan
- `debt` - apakah nasabah pernah melakukan gagal bayar pinjaman
- `total_income` - pendapatan bulanan
- `purpose` - tujuan mendapatkan pinjaman


In [6]:
df.shape

(21525, 12)

In [7]:
df.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Kita dapat melihat tiga masalah dari hasil eksplorasi pada data diatas:
1. Sebagian gelar dikolom education ditulis dalam huruf besar, sebagian dalam huruf kecil.
2. Data pada kolom days_employed ditulis menggunakan tanda (-) yang mengartikan negatif/kurang, yang seharusnya ditulis sebagai angka positif mengingat kolom ini menerangkan mengenai jumlah hari para nasabah telah bekerja.
3. Jumlah nilai kolom berbeda. Hal ini menandakan bahwa data yang kita miliki mengandung nilai yang hilang.

Untuk menyelesaikan masalah diatas diperlukan penyelidikan lebih lanjut.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Terdapat nilai yang hilang pada dua kolom dari data ini. Nilai yang hilang dapat dilihat pada kolom days_employed dan total_income yang mana jumlah nilainya lebih sedikit dibandingkan jumlah nilai pada kolom lainnya.

In [9]:
df.isna().sum()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Dengan melakukan filter untuk menemukan jumlah nilai hilang, dapat dilihat bahwa jumlah nilai yang hilang simetris pada kolom days_employed dan total_income. Namun, dari hasil filter diatas saja, tidak dapat disimpulkan bahwa nilai yang hilang pada kedua kolom berasal dari baris yang sama meskipun jumlah nilai yang hilang pada kedua baris simetris.

Diperlukan perhitungan lebih lanjut pada nilai yang hilang di semua baris dengan nilai yang hilang untuk memastikan bahwa sampel yang hilang memiliki ukuran yang sama.

In [10]:
df.loc[(df['days_employed'].isna()) & (df['total_income'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [11]:
missing_value_percentage_days_employed = (df['days_employed']).isna().sum() / len(df['days_employed']) * 100 
missing_value_percentage_total_income = (df['total_income']).isna().sum() / len(df['total_income']) * 100

print('Ini adalah persentase nilai lama bekerja yang hilang:')
print("{:.2f}%".format(missing_value_percentage_days_employed))
print()
print('Ini adalah persentase nilai penghasilan yang hilang:')
print("{:.2f}%".format(missing_value_percentage_total_income))

Ini adalah persentase nilai lama bekerja yang hilang:
10.10%

Ini adalah persentase nilai penghasilan yang hilang:
10.10%


**Kesimpulan sementara**

Dari hasil pemfilteran diatas - dengan kondisi dimana kolom 'days_employed' dan 'total_income' keduanya memiliki nilai yang hilang - dapat disimpulkan nilai yang hilang berasal dari baris yang sama pada kedua kolom. Hal ini dapat dilihat dalam tabel yang telah difilter, dimana jumlah baris dengan kondisi pemfilteran sesuai dengan jumlah nilai yang hilang.

Dan dilihat dari hasil persentase nilai yang hilang, nilai yang hilang ini merupakan bagian data yang cukup besar. Maka dari itu, perlu dilakukan pengisian untuk nilai yang hilang dengan cara memeriksa, apakah data yang hilang bisa jadi disebabkan oleh karakteristik nasabah tertentu, seperti jenis pekerjaan atau yang lainnya.

Untuk memeriksa hal ini, sebaiknya dilakukan perhitungan dengan pengelompokan untuk jumlah yang hilang pada kolom 'days_employed' dan 'total_income' dengan kolom 'income_type' dan 'education', mengingat data untuk jumlah hari kerja dan penghasilan erat kaitannya dengan jenis pekerjaan yang dilakukan nasabah serta tingkat pendidikannya. 

In [12]:
df_filtered = df.loc[(df['days_employed'].isna()) & (df['total_income'].isna())]
df_filtered['income_type'].value_counts().reset_index()

,index,income_type
0,employee,1105
1,business,508
2,retiree,413
3,civil servant,147
4,entrepreneur,1


In [13]:
df_filtered['education'].value_counts().reset_index()

,index,education
0,secondary education,1408
1,bachelor's degree,496
2,SECONDARY EDUCATION,67
3,Secondary Education,65
4,some college,55
5,Bachelor's Degree,25
6,BACHELOR'S DEGREE,23
7,primary education,19
8,Some College,7
9,SOME COLLEGE,7


Dari hasil pengelompokan data diatas, nilai yang hilang paling banyak ada pada jenis pekerjaan karyawan/pegawai dari kolom 'income_type', dan tingkat pendidikan sekolah menengah dari kolom 'education'.  

**Kemungkinan penyebab hilangnya nilai dalam data**

Pada pengujian data diatas belum bisa ditemukan kemungkinan alasan hilangnya nilai-nilai tersebut, sebab tidak ditemukan jumlah data yang simetris, serta cukup sulit menemukan pola yang ada pada kolom 'education' karena adanya kesalahan penulisan.

Untuk melakukan pemeriksaan apakah nilai yang hilang bersifat acak, perlu dilakukan perbandingan dari distribusi dataset yang asli dengan yang telah difilter. 

In [14]:
print('Data Statistik Table df :')
df.describe()

Data Statistik Table df :


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [15]:
print('Data Statistik table df yang telah difilter :')
df_filtered.describe()

Data Statistik table df yang telah difilter :


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2174.000000,0.0,2174.000000,2174.000000,2174.000000,2174.000000,0.0
mean,0.552438,NaN,43.632015,0.800828,0.975161,0.078197,NaN
std,1.469356,NaN,12.531481,0.530157,1.418220,0.268543,NaN
min,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,34.000000,0.250000,0.000000,0.000000,NaN
50%,0.000000,NaN,43.000000,1.000000,0.000000,0.000000,NaN
75%,1.000000,NaN,54.000000,1.000000,1.000000,0.000000,NaN
max,20.000000,NaN,73.000000,3.000000,4.000000,1.000000,NaN


**Kesimpulan sementara**

Dapat dilihat dari distribusi diatas, distribusi *dataset* yang asli mirip dengan distribusi tabel yang telah difilter. Hal ini berarti nilai pada dataset yang telah difilter dapat mewakili nilai dari dataset asli, yang membedakan hanyalah perhitungan nilai yang hilang pada kedua dataset dari kolom 'days_employed' dan 'total_income'. Hal ini juga memungkinkan kita untuk mengisi nilai yang hilang pada kedua kolom dengan nilai mean atau median dari keseluruhan nilai pada dataset, apabila hilangnya nilai-nilai tersebut terjadi secara acak.

Untuk menguji hal ini, perlu dilakukan pemeriksaan lebih lanjut apakah terdapat pola pada nilai yang hilang, mengingat belum ada kesimpulan tertentu yang dapat ditarik dari pengujian sebelumnya. Dengan mempertimbangkan status perkawinan nasabah juga dapat mempengaruhi pendapatan dan masa bekerja, selanjutnya *dataset* akan diuji berdasarkan kolom 'family_status. 

In [16]:
df_filtered['family_status'].value_counts().reset_index()


,index,family_status
0,married,1237
1,civil partnership,442
2,unmarried,288
3,divorced,112
4,widow / widower,95


**Kesimpulan sementara**

Dari pengelompokan data berdasarkan status perkawinan untuk nilai yang hilang diatas, juga tidak ditemukan adanya pola tertentu yang dapat disimpulkan, kemungkinan nilai-nilai tersebut hilang secara acak/kebetulan.  

**Kesimpulan**

Setelah memeriksa nilai yang hilang berdasarkan ketergantungannya pada indikator dari nilai didalam tiga kolom lain, yakni : jenis pekerjaan, tingkat pendidikan, dan status perkawinan, tidak ditemukan pola tertentu yang menjadi penyebab hilangnya nilai-nilai tersebut tidak secara acak. Mempertimbangkan ketiga kolom ini berkaitan erat untuk memperoleh data mengenai masa bekerja dan pendapatan, penting untuk mengidentifikasikan karakteristik tertentu nasabah melalui ketiga indikator ini, untuk menentukan bagaimana nilai yang hilang akan diisi.

Selain itu, melihat perbandingan statistik dari dataset asli dan dataset yang telah difilter tidak ditemukan adanya perbedaan yang signifikan, untuk mengatasi nilai-nilai yang hilang dapat diisi dengan mean atau median dari masing-masing kolom, mempertimbangkan kategori dari nilai yang hilang merupakan data numerik.

Tahapan selanjutnya, yaitu mentransformasi data, untuk mengatasi berbagai jenis masalah yang ditemukan seperti: duplikat, pencatatan yang berbeda, sumber data yang salah, dan nilai yang hilang.

## Transformasi data

Memeriksa masalah pada setiap kolom.

Memperbaiki masalah penulisan tentang data pendidikan. Penulisan ditulis sebagian dengan huruf besar dan sebagian dengan huruf kecil. Untuk memperbaiki masalah ini, semua penulisan dibuat menjadi huruf kecil.  

In [17]:
df['education'].sort_values().unique()

array(["BACHELOR'S DEGREE", "Bachelor's Degree", 'GRADUATE DEGREE',
       'Graduate Degree', 'PRIMARY EDUCATION', 'Primary Education',
       'SECONDARY EDUCATION', 'SOME COLLEGE', 'Secondary Education',
       'Some College', "bachelor's degree", 'graduate degree',
       'primary education', 'secondary education', 'some college'],
      dtype=object)

In [18]:
df['education'] = df['education'].str.lower()


In [19]:
df['education'].sort_values().unique()


array(["bachelor's degree", 'graduate degree', 'primary education',
       'secondary education', 'some college'], dtype=object)

Memeriksa data kolom `children`. 

In [20]:
df['children'].sort_values().unique()

array([-1,  0,  1,  2,  3,  4,  5, 20], dtype=int64)

Data dikolom `children` memberikan informasi mengenai jumlah anak yang dimiliki nasabah. Namun terdapat kejanggalan pada penulisan data dimana salah satu angka ditulis sebagai angka negatif `-1`, dan salah satu nilai lainnya adalah angka `20`. Kemungkinan hal ini terjadi karena kesalahan saat penginputan data/pengetikan, karena tidak masuk akal jumlah anak ditulis sebagai angka negatif atau jumlah 20 anak tidaklah wajar. Mengingat presentasi kesalahan ini cukup kecil, angka yang negatif ini akan diganti menjadi positif, dan angka 20 akan diubah menjadi angka 2 untuk memperbaiki kesalahan ini.

In [21]:
count_minus1 = df.loc[df['children'] == -1, 'children'].count() 
total_count = len(df['children'])
percentage_minus1 = (count_minus1 / total_count) * 100

print('Persentase nilai -1 : {:.2f}%'.format(percentage_minus1))


Persentase nilai -1 : 0.22%


In [22]:
count_20 = df.loc[df['children'] == 20, 'children'].count() 
total_count = len(df['children'])
percentage_minus1 = (count_20 / total_count) * 100

print('Persentase nilai 20 : {:.2f}%'.format(percentage_minus1))

Persentase nilai 20 : 0.35%


In [23]:
df.loc[df['children'] == -1, 'children'] = 1
df.loc[df['children'] == 20, 'children'] = 2

df['children'].sort_values().unique()


array([0, 1, 2, 3, 4, 5], dtype=int64)

Memeriksa data dalam kolom the `days_employed`. 
Masalah yang terdapat pada kolom ini yaitu penulisan angka ditulis menjadi angka negatif. Sebelum memperbaiki masalah ini, perlu dilakukan perhitungan terlebih dahulu untuk memeriksa presentasi dari jumlah kesalahan ini.

In [24]:
df['days_employed'].head(15)

0      -8437.673028
1      -4024.803754
2      -5623.422610
3      -4124.747207
4     340266.072047
5       -926.185831
6      -2879.202052
7       -152.779569
8      -6929.865299
9      -2188.756445
10     -4171.483647
11      -792.701887
12              NaN
13     -1846.641941
14     -1844.956182
Name: days_employed, dtype: float64

In [25]:
negative_percentage = (df['days_employed'] < 0).mean() * 100
print("Persentase angka negatif dalam kolom 'days_employed': {:.2f}%".format(negative_percentage))

Persentase angka negatif dalam kolom 'days_employed': 73.90%


Jumlah data yang bermasalah tinggi, dapat dilihat dari hasil presentasinya. Hal tersebut mungkin disebabkan oleh beberapa masalah teknis saat penginputan data. Mengingat kolom`days_employed` memberikan informasi mengenai lama hari bekerja para nasabah, informasi yang benar seharusnya dituliskan sebagai angka positif. Untuk itu, data bermasalah yang dituliskan sebagai angka negatif akan diubah menjadi angka positif, mengingat semua baris yang bermasalah ini tidak bisa dihapus.

In [26]:
df['days_employed'] = df['days_employed'].abs()

df['days_employed'].head(15)

0       8437.673028
1       4024.803754
2       5623.422610
3       4124.747207
4     340266.072047
5        926.185831
6       2879.202052
7        152.779569
8       6929.865299
9       2188.756445
10      4171.483647
11       792.701887
12              NaN
13      1846.641941
14      1844.956182
Name: days_employed, dtype: float64

Memerika masalah pada kolom usia nasabah. Terdapat kesalahan pada salah satu data dimana usia nasabah ditulis sebagai 0 dalam kolom ini, dapat disimpulkan angka usia ini tidak masuk akal. Sebelum memperbaiki masalah ini, perlu dilakukan perhitungan terlebih dahulu untuk memeriksa presentasi dari jumlah kesalahan ini.

In [27]:
df['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75], dtype=int64)

In [28]:
count_0 = df['dob_years'].value_counts()[0]
total_count = len(df['dob_years'])

percentage_0 = (count_0 / total_count) * 100

print('Persentase nilai 0: {:.2f}%'.format(percentage_0))

Persentase nilai 0: 0.47%


In [29]:
df_dob_years_statistics = df['dob_years'].agg(['mean', 'median'])

mean_value = round(df_dob_years_statistics['mean'])
median_value = round(df_dob_years_statistics['median'])
print('Mean :', mean_value)
print('Median :', median_value)

Mean : 43
Median : 42


In [30]:
df['dob_years'] = df['dob_years'].replace(0, mean_value)

print(df['dob_years'])

0        42
1        36
2        33
3        32
4        53
         ..
21520    43
21521    67
21522    38
21523    38
21524    40
Name: dob_years, Length: 21525, dtype: int64


In [31]:
df['dob_years'].sort_values().unique()

array([19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75], dtype=int64)

Presentasi dari jumlah kesalahan ini cukup kecil, namun kesalahan ini tetap akan menyulitkan proses analisa yang diperlukan nantinya, karena usia nasabah 0 tahun tidaklah masuk akal. Untuk mengganti nilah bermasalah ini, dibutuhkan nilai mean dan median dari kolom tersebut. Dan dapat dilihat dari kedua hasilnya (mean, dan median) sama. Hasil dari perhitungan rata-rata dapat digunakan disini untuk mengganti angka yang bermasalah. 

Memeriksa kolom `family_status`. Masalah yang muncul pada data kolom ini adalah penulisan widow / widower yang berulang, meskipun memiliki arti yang sama. Untuk memperbaiki hal ini, penulisannya diubah menjadi `widow`. 

In [32]:
df['family_status'].sort_values().unique()


array(['civil partnership', 'divorced', 'married', 'unmarried',
       'widow / widower'], dtype=object)

In [33]:
df.loc[df['family_status'] == 'widow / widower', 'family_status'] = 'widow'

In [34]:
df['family_status'].sort_values().unique()

array(['civil partnership', 'divorced', 'married', 'unmarried', 'widow'],
      dtype=object)

Memeriksa kolom `gender`. Terdapat kesalahan pada salah satu penulisan data pada kolom gender, yaitu 'XNA'. Hal ini merupakan sebuah anomali pada data kolom ini, oleh karena itu baris dengan nilai 'XNA' akan dihapus. 

In [35]:
df['gender'].sort_values().unique()

array(['F', 'M', 'XNA'], dtype=object)

In [36]:
df.drop(df[df['gender'] == 'XNA'].index, inplace=True)

In [37]:
df['gender'].sort_values().unique()

array(['F', 'M'], dtype=object)

Memeriksa kolom `income_type`. Nilai yang dimuat di dalam kolom ini memberikan informasi mengenai jenis pekerjaan nasabah, dan dapat dilihat tidak terdapat ada masalah yang muncul pada data kolom ini.

In [38]:
df['income_type'].sort_values().unique()

array(['business', 'civil servant', 'employee', 'entrepreneur',
       'paternity / maternity leave', 'retiree', 'student', 'unemployed'],
      dtype=object)

Memeriksa duplikat di dalam data. Ditemukan terdapat 71 duplikat didalam data. Mengingat kesalahan penulisan pada setiap kolom sudah diperbaiki sebelumnya, data duplikat dapat langsung dihapus untuk menangani masalah ini.

In [39]:
df.duplicated().sum()

71

In [40]:
df = df.drop_duplicates().reset_index(drop=True)

In [41]:
df.duplicated().sum()

0

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      19350 non-null  float64
 11  purpose           21453 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Diatas merupakan hasil dataset yang telah diperbaiki untuk kesalahan penulisannya, serta data duplikat yang ada telah dihapus. Terdapat pengurangan baris sebanyak 71 baris dari data awal, yang mana jumlah ini sama dengan jumlah data duplikat yang telah dihapus. 


# Bekerja dengan nilai yang hilang

Menggunakan nama kategori yang panjang dalam pemrosesan data dapat menyebabkan error, maka dari itu, penting menggunakan *dictionary* untuk beberapa nilai yang memiliki ID, yang ada dikolom : `education_id` dan `family_status_id`.

In [43]:
df_edu = dict(zip(df.education_id, df.education))
df_edu

{0: "bachelor's degree",
 1: 'secondary education',
 2: 'some college',
 3: 'primary education',
 4: 'graduate degree'}

In [44]:
df_fam = dict(zip(df.family_status_id, df.family_status))
df_fam

{0: 'married',
 1: 'civil partnership',
 2: 'widow',
 3: 'divorced',
 4: 'unmarried'}

### Memperbaiki nilai yang hilang di `total_income`

Terdapat sejumlah nilai yang hilang dikolom `total_income` dan kolom `days_employed`. Pada bagian ini kolom dengan nilai yang hilang yang akan diperbaiki terlebih dahulu yaitu kolom `total_income`, dengan mengisi nilai yang sesuai/mirip dengan clusternya. 

Mengingat jumlah pendapatan dapat terpengaruh oleh usia & jenis pekerjaan dari para nasabah, untuk itu, nilai yang hilang pada kolom ini dapat diisi dengan hasil rata-rata atau median dari kolom `total_income` yang telah dikelompokan berdasarkan usia & jenis pekerjaan nasabah.

Sebelumnya, kategori usia untuk nasabah perlu dikelompokan terlebih dahulu, untuk  membantu menghitung total nilai rata-rata dan median pendapatan.

In [45]:
def age_category(dob_years):
    if dob_years <= 35:
        return '19-35'
    if dob_years <= 50:
        return '36-50'
    if dob_years <= 65:
        return '51-65'
    else:
        return '66+'

In [46]:
print(age_category(40)) 

36-50


In [47]:
df['age_category'] = df['dob_years'].apply(age_category)
df['age_category'].value_counts()

36-50    8227
19-35    6582
51-65    5942
66+       702
Name: age_category, dtype: int64

In [48]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,36-50
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,36-50
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,19-35
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,19-35
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,51-65
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,19-35
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,36-50
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,36-50
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,19-35
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,36-50


Pada dataset ini kolom kategori usia nasabah telah ditambahkan, untuk mempermudah perhitungan mean dan median pendapatan yang akan dikelompokan berdasarkan kategori usia ini. Namun, untuk menghitung nilai mean dan median dengan tepat, dataset tanpa nilai yang hilang perlu dibuat terlebih dahulu.

Dataset baru yang hanya memuat data tanpa nilai yang hilang. Data ini akan digunakan untuk menghitung mean dan median dari kolom `total_income`.

In [49]:
df_clean = df.dropna().reset_index(drop=True)
df_clean.info()
df_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19350 entries, 0 to 19349
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          19350 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         19350 non-null  int64  
 3   education         19350 non-null  object 
 4   education_id      19350 non-null  int64  
 5   family_status     19350 non-null  object 
 6   family_status_id  19350 non-null  int64  
 7   gender            19350 non-null  object 
 8   income_type       19350 non-null  object 
 9   debt              19350 non-null  int64  
 10  total_income      19350 non-null  float64
 11  purpose           19350 non-null  object 
 12  age_category      19350 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 1.9+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,36-50
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,36-50
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,19-35
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,19-35
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,51-65


In [50]:
df_grouped = df_clean.groupby(['age_category', 'income_type']).agg(
    {'total_income': ['mean', 'median']}) 

df_grouped

total_income            
                                                  mean      median
age_category income_type                                          
19-35        business                     30632.221433  26510.1310
             civil servant                26969.345533  24368.0150
             employee                     25080.209850  22261.7270
             entrepreneur                 79866.103000  79866.1030
             retiree                      18893.982687  15378.5635
             student                      15712.260000  15712.2600
             unemployed                    9593.119000   9593.1190
36-50        business                     33851.437343  28773.8005
             civil servant                27718.403926  23683.4620
             employee                     26205.601037  23087.8590
             paternity / maternity leave   8612.661000   8612.6610
             retiree                      25652.266036  22144.2240
             unemployed                   32435.602000  32435.6020
51-65        business                     32410.103403  27452.7460
             civil servant                26967.495276  23333.3000
             employee                     26415.051987  22777.3180
             retiree                      22129.451966  19306.1320
66+          business                     33470.062000  29314.4045
             civil servant                30992.299000  26089.6870
             employee                     26185.024870  24643.1985
             retiree                      19666.105943  17098.2030

Dapat disimpulkan dari perhitungan nilai mean dan median dari kolom `total_income` yang dikelompokan berdasarkan kategori usia & jenis pekerjaan diatas, bahwa nilai mean dan median memiliki perbedaan nilai yang cukup signifikan pada sebagian kategori. Hal ini menunjukan, adanya outlier pada nilai dikolom `total_income`. Maka dari itu, nilai yang lebih baik digunakan untuk mengisi nilai yang hilang dikolom pendapatan adalah nilai median, mengingat nilai median menghasilkan nilai yang lebih akurat dibandingkan nilai mean, apabila terdapat outlier didalam data. 

In [51]:
def fillna_total_income_by_median(df):
    
    median_by_age_income_type = df.groupby(['age_category', 'income_type'])['total_income'].median()

    for (age_category, income_type), median in median_by_age_income_type.items():
        df.loc[(df['age_category'] == age_category) & (df['income_type'] == income_type) & (df['total_income'].isnull()), 'total_income'] = median

    return df

In [52]:
df = fillna_total_income_by_median(df)

In [53]:
df.isna().sum()

children               0
days_employed       2103
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           1
purpose                0
age_category           0
dtype: int64

Memeriksa keberadaan nilai yang hilang dikolom `total_income`. Karena masih terdapat satu baris dengan nilai yang hilang dikolom ini, perlu dilakukan pemeriksaan dengan melihat baris tersebut.

In [54]:
df.loc[(df['total_income'].isnull())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
5931,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,51-65


In [55]:
df.loc[(df['age_category'] == '51-65') & (df['income_type'] == 'entrepreneur')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
5931,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,51-65


Setelah melakukan pemeriksaan pada satu baris dengan nilai yang hilang pada kolom `total_income`, ternyata hanya terdapat satu baris data dari pengkategorian kondisi yang melalui pemfilteran berdasarkan usia dan jenis pekerjaan.

Untuk menangani hal ini, nilai yang hilang akan diperbaiki secara manual, dengan menggunakan nilai median yang dikelompokan berdasarkan kategori usia nasabah saja. 

In [56]:
df['total_income'] = df['total_income'].fillna(df_clean[df_clean['age_category'] == '51-65']['total_income'].median())

df.isna().sum()

children               0
days_employed       2103
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_category           0
dtype: int64

Seluruh nilai yang hilang pada kolom `total_income` telah ditangani. 

Jumlah total nilai di kolom `total_income` sudah sesuai dengan jumlah nilai di kolom lain.

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  float64
 11  purpose           21453 non-null  object 
 12  age_category      21453 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


###  Memperbaiki nilai di `days_employed`

Serupa dengan kolom `total_income`, mengingat jumlah hari bekerja dapat terpengaruh oleh usia & jenis pekerjaan dari para nasabah, untuk itu, nilai yang hilang pada kolom ini dapat diisi dengan hasil rata-rata atau median dari kolom `days_employed` yang telah dikelompokan berdasarkan usia & jenis pekerjaan nasabah.

In [58]:
mean_by_age_income_type = df_clean.groupby(['age_category', 'income_type'])['days_employed'].mean()

df_mean_by_age_income_type = mean_by_age_income_type.reset_index(name='mean_days_employed')

df_mean_by_age_income_type.pivot_table(index='income_type', 
                                         columns='age_category', 
                                         values='mean_days_employed', 
                                         aggfunc='mean')

age_category,19-35,36-50,51-65,66+
income_type,,,,
business,1436.512015,2333.773426,3001.467627,3725.387000
civil servant,1985.858462,3882.803044,4841.244126,4145.742201
employee,1533.090064,2587.605818,3332.145342,4092.413329
entrepreneur,520.848083,NaN,NaN,NaN
paternity / maternity leave,NaN,3296.759962,NaN,NaN
retiree,361996.763055,365477.270127,364844.835633,365640.450983
student,578.751554,NaN,NaN,NaN
unemployed,337524.466835,395302.838654,NaN,NaN


In [59]:
median_by_age_income_type = df_clean.groupby(['age_category', 'income_type'])['days_employed'].median()

df_median_by_age_income_type = median_by_age_income_type.reset_index(name='median_days_employed')

df_median_by_age_income_type.pivot_table(index='income_type', 
                                         columns='age_category', 
                                         values='median_days_employed', 
                                         aggfunc='median', 
                                         margins=True)


age_category,19-35,36-50,51-65,66+,All
income_type,,,,,
business,1095.703678,1832.564621,2113.077981,2318.709538,1972.821301
civil servant,1756.744770,3450.294508,3826.493631,4137.331615,3638.394069
employee,1199.103335,1858.785090,2280.548023,2830.361431,2069.666556
entrepreneur,520.848083,NaN,NaN,NaN,520.848083
paternity / maternity leave,NaN,3296.759962,NaN,NaN,3296.759962
retiree,361942.896424,366457.872613,364914.419761,365934.432696,365424.426228
student,578.751554,NaN,NaN,NaN,578.751554
unemployed,337524.466835,395302.838654,NaN,NaN,366413.652744
All,1199.103335,3373.527235,3053.520827,3483.846523,2830.361431


Dapat disimpulkan dari perhitungan nilai mean dan median dari kolom `days_employed` yang dikelompokan berdasarkan kategori usia & jenis pekerjaan diatas, bahwa nilai mean dan median memiliki perbedaan nilai yang cukup signifikan pada sebagian kategori. Hal ini menunjukan, adanya outlier pada nilai dikolom `days_employed`. Maka dari itu, nilai yang lebih baik digunakan untuk mengisi nilai yang hilang dikolom pendapatan adalah nilai median, mengingat nilai median menghasilkan nilai yang lebih akurat dibandingkan nilai mean, apabila terdapat outlier didalam data. 

In [60]:
def fillna_days_employed_by_median(df):
    
    median_by_age_income_type = df.groupby(['age_category', 'income_type'])['days_employed'].median()

    for (age_category, income_type), median in median_by_age_income_type.items():
        df.loc[(df['age_category'] == age_category) & (df['income_type'] == income_type) & (df['days_employed'].isnull()), 'days_employed'] = median

    return df


In [61]:
df = fillna_days_employed_by_median(df)

In [62]:
df.isna().sum()

children            0
days_employed       1
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_category        0
dtype: int64

In [63]:
df.loc[(df['days_employed'].isnull())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
5931,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,21650.9475,buy residential real estate,51-65


In [64]:
df.loc[(df['age_category'] == '51-65') & (df['income_type'] == 'entrepreneur')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
5931,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,21650.9475,buy residential real estate,51-65


Setelah melakukan pemeriksaan pada satu baris dengan nilai yang hilang pada kolom `days_employed`,  terdapat satu baris data dari pengkategorian kondisi yang melalui pemfilteran berdasarkan usia dan jenis pekerjaan. Ini merupakan baris yang sama dengan nilai Nan pada pemeriksaan `total_income` sebelumnya. 

Untuk menangani hal ini, nilai yang hilang akan diperbaiki secara manual, dengan menggunakan nilai median yang dikelompokan berdasarkan kategori usia nasabah saja. 

In [65]:
df['days_employed'] = df['days_employed'].fillna(df_clean[df_clean['age_category'] == '51-65']['days_employed'].median())

df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_category        0
dtype: int64

Jumlah total nilai di kolom `days_employed` sudah sesuai dengan jumlah nilai di kolom lain.

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     21453 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  float64
 11  purpose           21453 non-null  object 
 12  age_category      21453 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


## Pengkategorian Data

Untuk menjawab pertanyaan dan menguji hipotesis, data perlu dikategorikan terlebih dahulu. Terdapat data teks dan data numerik yang perlu dikategorikan untuk menjawab pertanyaan dan menguji hipotesis, yaitu : kolom `purpose`, kolom `children`, dan kolom '`total_income`. Ketiga kolom ini dibutuhkan karena nilai datanya berkaitan untuk menguji setiap hipotesis.

Kolom pertama yang akan dikategorikan adalah kolom `purpose`.

In [67]:
df.purpose

0          purchase of the house
1                   car purchase
2          purchase of the house
3        supplementary education
4              to have a wedding
                  ...           
21448       housing transactions
21449          purchase of a car
21450                   property
21451          buying my own car
21452               to buy a car
Name: purpose, Length: 21453, dtype: object

Memeriksa nilai unik kolom `purpose`.

In [68]:
df['purpose'].sort_values().unique()

array(['building a property', 'building a real estate',
       'buy commercial real estate', 'buy real estate',
       'buy residential real estate', 'buying a second-hand car',
       'buying my own car', 'buying property for renting out', 'car',
       'car purchase', 'cars', 'construction of own property',
       'education', 'getting an education', 'getting higher education',
       'going to university', 'having a wedding', 'housing',
       'housing renovation', 'housing transactions', 'profile education',
       'property', 'purchase of a car', 'purchase of my own house',
       'purchase of the house', 'purchase of the house for my family',
       'real estate transactions', 'second-hand car purchase',
       'supplementary education', 'to become educated', 'to buy a car',
       'to get a supplementary education', 'to have a wedding',
       'to own a car', 'transactions with commercial real estate',
       'transactions with my real estate', 'university education',
       'we

Setelah identifikasi berdasarkan nilai uniknya, terdapat empat kelompok utama dalam kolom ini yang dapat dikategorikan.  

Pengkategorian data dikolom `purpose` menjadi empat kelompok sebagai kolom baru, `purpose_category` :

In [69]:
def purpose_category(purpose):

    if 'property' in purpose:
        return 'real estate'
    elif 'real estate' in purpose:
        return 'real estate'
    elif 'hous' in purpose:
        return 'real estate'
    elif 'university' in purpose:
        return 'education'
    elif 'edu' in purpose:
        return 'education'
    elif 'car' in purpose:
        return 'car'
    elif 'wedd' in purpose:
        return 'wedding'

In [70]:
print(purpose_category('to become educated'))

education


In [71]:
df['purpose_category'] = df['purpose'].apply(purpose_category)
df['purpose_category'].value_counts()

real estate    10810
car             4306
education       4013
wedding         2324
Name: purpose_category, dtype: int64

Data numerik yang akan dikategorikan adalah kolom `children` dan kolom `total_income`:

In [72]:
df[['children', 'total_income']].head(10)

,children,total_income
0,1,40620.102
1,1,17932.802
2,0,23341.752
3,3,42820.568
4,0,25378.572
5,0,40922.170
6,0,38484.156
7,0,21731.829
8,2,15337.093
9,0,23108.150


In [73]:
df[['children', 'total_income']].describe()

,children,total_income
count,21453.000000,21453.000000
mean,0.480585,26449.648385
std,0.756079,15707.133484
min,0.000000,3306.762000
25%,0.000000,17129.945000
50%,0.000000,23087.859000
75%,1.000000,31327.922000
max,5.000000,362496.645000


Kolom pertama yang akan dikategorikan adalah kolom `children`, yang akan dikategorikan menjadi tiga kelompok : yang tidak memiliki anak, yang memilik satu/dua anak, dan yang memiliki lebih dari 3 anak. Rentang jumlah anak dikelompokan seperti ini, untuk mempermudah pengelompokan secara umum jumlah anak para nasabah. Kategori jumlah anak akan ditambahkan sebagai kolom baru : `children_kategory`.    

In [74]:
def children_category(children):
    if children == 0:
        return 'childless'
    if children <= 2:
        return '1-2 children'
    else:
        return '3+ children'

print(children_category(1))

1-2 children


In [75]:
df['children_category'] = df['children'].apply(children_category)

In [76]:
df['children_category'].value_counts()  

childless       14090
1-2 children     6983
3+ children       380
Name: children_category, dtype: int64

Pengelompokan selanjutanya untuk data numerik yaitu, kolom `total_income`. Nilai pada kolom ini akan dikategorikan menjadi lima kelompok yang berbeda, agar mencakup seluruh rentang nilainya dan mempermudah untuk pengujian hipotesis. Kategori pendapatan ini akan ditambahkan sebagai kolom baru : `income_category`.

In [77]:
def income_category(total_income):
    if total_income <= 10000:
        return 'less than 10.000'
    elif total_income <= 50000:
        return '10.000-50.000'
    elif total_income <= 100000:
        return '50.000-100.000'
    elif total_income <= 200000:
        return '100.000-200.000'
    else:
        return '200.000+'

print(income_category(25000))

10.000-50.000


In [78]:
df['income_category'] = df['total_income'].apply(income_category)
df['income_category'].value_counts()

10.000-50.000       19207
50.000-100.000       1221
less than 10.000      926
100.000-200.000        88
200.000+               11
Name: income_category, dtype: int64

In [79]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,purpose_category,children_category,income_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,36-50,real estate,1-2 children,10.000-50.000
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,36-50,car,1-2 children,10.000-50.000
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,19-35,real estate,childless,10.000-50.000
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,19-35,education,3+ children,10.000-50.000
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,51-65,wedding,childless,10.000-50.000


Pada tabel data yang telah diperbaharui ini , ketiga kolom kategori telah ditambahkan.

## Memeriksa hipotesis


**Apakah terdapat korelasi antara memiliki anak dengan probabilitas melakukan gagal bayar pinjaman?**

In [80]:
debt_by_children_group = df.groupby('children_category')['debt'].sum().sort_values(ascending=False).reset_index()
debt_by_children_group

,children_category,debt
0,childless,1063
1,1-2 children,647
2,3+ children,31


In [81]:
def calculate_percentage(children_category):
    
    filtered_data = debt_by_children_group[debt_by_children_group['children_category'] == children_category]
    
    total_debt = filtered_data['debt'].sum()
    
    percentage = (total_debt / debt_by_children_group['debt'].sum()) * 100
    
    return percentage

result = calculate_percentage(children_category)

print(f"Persentase gagal bayar nasabah yang tidak memiliki anak: {calculate_percentage('childless'):.2f}%")
print(f"Persentase gagal bayar nasabah yang memiliki 1 atau 2 anak: {calculate_percentage('1-2 children'):.2f}%")
print(f"Persentase gagal bayar nasabah yang memiliki lebih dari 3 anak: {calculate_percentage('3+ children'):.2f}%") 


Persentase gagal bayar nasabah yang tidak memiliki anak: 61.06%
Persentase gagal bayar nasabah yang memiliki 1 atau 2 anak: 37.16%
Persentase gagal bayar nasabah yang memiliki lebih dari 3 anak: 1.78%


**Kesimpulan**

Untuk menjawab hipotesis pertama, dilakukan perhitungan presentase nasabah yang pernah gagal bayar, dari tiap kategori jumlah anak. Dapat dilihat dari hasilnya hipotesis pertama tidak dapat diterima, karena dari hasil presentase jumlah nasabah yang melakukan gagal bayar terbanyak adalah nasabah yang tidak memiliki anak, sebanyak 61.06%, dan ini berbanding terbalik dengan nasabah yang memiliki lebih dari 3 anak yang mana presentase gagal bayarnya hanya 1.78%. 


**Apakah terdapat korelasi antara status keluarga dengan probabilitas melakukan gagal bayar pinjaman?**

In [82]:
debt_by_family_status = df.groupby('family_status')['debt'].sum().sort_values(ascending=False).reset_index()
debt_by_family_status

,family_status,debt
0,married,931
1,civil partnership,388
2,unmarried,274
3,divorced,85
4,widow,63


In [83]:
def calculate_percentage(family_status):
    
    filtered_data = debt_by_family_status[debt_by_family_status['family_status'] == family_status]
    
    total_debt = filtered_data['debt'].sum()
    
    percentage = (total_debt / debt_by_family_status['debt'].sum()) * 100
    
    return percentage

print(f"Persentase gagal bayar nasabah yang sudah menikah: {calculate_percentage('married'):.2f}%")
print(f"Persentase gagal bayar nasabah dengan status civil partnership: {calculate_percentage('civil partnership'):.2f}%")
print(f"Persentase gagal bayar nasabah yang tidak menikah: {calculate_percentage('unmarried'):.2f}%")
print(f"Persentase gagal bayar nasabah dengan status bercerai: {calculate_percentage('divorced'):.2f}%")
print(f"Persentase gagal bayar nasabah dengan status janda/duda: {calculate_percentage('widow'):.2f}%")

Persentase gagal bayar nasabah yang sudah menikah: 53.48%
Persentase gagal bayar nasabah dengan status civil partnership: 22.29%
Persentase gagal bayar nasabah yang tidak menikah: 15.74%
Persentase gagal bayar nasabah dengan status bercerai: 4.88%
Persentase gagal bayar nasabah dengan status janda/duda: 3.62%


**Kesimpulan**

Untuk hipotesis kedua, perhitungan presentase gagal bayar dilakukan berdasarkan tiap nilai di `family_status`. Dari data diatas, dapat diasumsikan status pernikahan cukup mempengaruhi kemampuan nasabah dalam membayar pinjaman, dimana nasabah dengan status yang sudah menikah memiliki presentase terbesar yang pernah melakukan gagal bayar. Presentase gagal bayar nasabah yang sudah menikah mencapai 53.48% dari lima kategori di `family_status`, dan nasabah dengan status janda/duda merupakan nasabah dengan presentase gagal bayar terendah, yaitu 3.62% saja.  


**Apakah terdapat korelasi antara tingkat pendapatan dengan probabilitas melakukan gagal bayar pinjaman?**

In [84]:
debt_by_income_group = df.groupby('income_category')['debt'].sum().sort_values(ascending=False).reset_index()
debt_by_income_group


,income_category,debt
0,10.000-50.000,1591
1,50.000-100.000,86
2,less than 10.000,58
3,100.000-200.000,5
4,200.000+,1


In [85]:
def calculate_percentage(income_category):
    
    filtered_data = debt_by_income_group[debt_by_income_group['income_category'] == income_category]
    
    total_debt = filtered_data['debt'].sum()
    
    percentage = (total_debt / debt_by_income_group['debt'].sum()) * 100
    
    return percentage

result = calculate_percentage(income_category)

print(f"Persentase gagal bayar nasabah dengan penghasilan 10.000-50.000: {calculate_percentage('10.000-50.000'):.2f}%")
print(f"Persentase gagal bayar nasabah dengan penghasilan 50.000-100.000: {calculate_percentage('50.000-100.000'):.2f}%")
print(f"Persentase gagal bayar nasabah dengan penghasilan kurang dari 10.000: {calculate_percentage('less than 10.000'):.2f}%")
print(f"Persentase gagal bayar nasabah dengan penghasilan 100.000-200.000: {calculate_percentage('100.000-200.000'):.2f}%")
print(f"Persentase gagal bayar nasabah dengan penghasilan lebih dari 200.000: {calculate_percentage('200.000+'):.2f}%")

Persentase gagal bayar nasabah dengan penghasilan 10.000-50.000: 91.38%
Persentase gagal bayar nasabah dengan penghasilan 50.000-100.000: 4.94%
Persentase gagal bayar nasabah dengan penghasilan kurang dari 10.000: 3.33%
Persentase gagal bayar nasabah dengan penghasilan 100.000-200.000: 0.29%
Persentase gagal bayar nasabah dengan penghasilan lebih dari 200.000: 0.06%


**Kesimpulan**

Hipotesis ketiga dilakukan dengan menghitung presentase gagal bayar nasabah berdasarkan tingkat pendapatannya. Dari hasil pengamatan data diatas, hipotesis ketiga dapat diterima, sebab nasabah dengan penghasilan terbesar - lebih dari 200.000 - memiliki presentase gagal bayar terendah sebesar 0.06%, dan nasabah dengan presentae gagal bayar tertinggi yaitu nasabah dengan rentang penghasilan 10.000-50.000, sebanyak 91.38%. Pada data, ini memang bukanlah nasabah dengan penghasilan terendah, namun masih dapat digolongkan kedalam nasabah dengan penghasilan yang rendah. 

**Bagaimana tujuan kredit memengaruhi persentase gagal bayar?**

In [86]:
debt_by_purpose = df.groupby('purpose_category')['debt'].sum().sort_values(ascending=False).reset_index()
debt_by_purpose

,purpose_category,debt
0,real estate,782
1,car,403
2,education,370
3,wedding,186


In [87]:
def calculate_percentage(purpose_category):
    
    filtered_data = debt_by_purpose[debt_by_purpose['purpose_category'] == purpose_category]
    
    total_debt = filtered_data['debt'].sum()
    
    percentage = (total_debt / debt_by_purpose['debt'].sum()) * 100
    
    return percentage

result = calculate_percentage(purpose_category)

print(f"Persentase gagal bayar nasabah dengan tujuan pembelian real estate: {calculate_percentage('real estate'):.2f}%")
print(f"Persentase gagal bayar nasabah dengan tujuan pembelian mobil: {calculate_percentage('car'):.2f}%")
print(f"Persentase gagal bayar nasabah dengan tujuan biaya pendidikan: {calculate_percentage('education'):.2f}%")
print(f"Persentase gagal bayar nasabah dengan tujuan biaya pernikahan: {calculate_percentage('wedding'):.2f}%")

Persentase gagal bayar nasabah dengan tujuan pembelian real estate: 44.92%
Persentase gagal bayar nasabah dengan tujuan pembelian mobil: 23.15%
Persentase gagal bayar nasabah dengan tujuan biaya pendidikan: 21.25%
Persentase gagal bayar nasabah dengan tujuan biaya pernikahan: 10.68%


**Kesimpulan**

Hipotesis terakhir diuji berdasarkan tujuan nasabah melakukan kredit. Dari hasil analisa, presentase nasabah dengan tujuan pembelian real estate merupakan presentase tertinggi yang pernah melakukan gagal bayar, sebesar 44.92%, dan yang terendah merupakan nasabah dengan tujuan biaya pernikahan, sebesar 10.68%. Hal ini menunjukan hipotesis terakhir dapat diterima, sebab tujuan nasabah mengajukan kredit cukup mempengaruhi kemampuan nasabah melakukan pembayaran.


# Kesimpulan umum 

Proyek ini bertujuan untuk menganalisa data kemampuan nasabah dalam melunasi pembayaran kredit, berdasarkan beberapa kategori. Sebelum memproses data, dilakukan pra-proses data terlebih dahulu untuk memperbaiki kesalahan penulisan, menghapus duplikat, dan mengisi nilai yang hilang. Nilai yang hilang terdapat di dua kolom, yaitu : `days_employed` dan `total_income`, diperbaiki dengan cara mengitung nilai mean dan median dari cluster yang mirip dan kolom yang mempengaruhi langsung data pada kedua kolom dengan nilai yan hilang ini. Kolom yang digunakan untuk menghitung nilai mean dan median adalah kolom `income_type` dan `age_category` sebab nilai pada kedua kolom ini berkaitan erat dengan data pada kedua kolom dengan nilai yang hilang. Dan pada tahap pengisian nilai yang hilang, nilai yang dipilih adalah hasil median, sebab ditemukan perbedaan hasil yang cukup signifikan dari hasil mean dan median, dimana dapat disimpulkan bahwa terdapat outlier pada data, sehingga nilai median dianggap lebih akurat.

Menguji empat hipotesis:

1. Terdapat hubungan antara memiliki anak dan probabilitas seseorang melakukan gagal bayar pinjaman.
Hipotesis ini tidak dapat diterima, karena memiliki anak tidak membuat probabilitas nasabah melakukan gagal bayar pinjam. Dari hasil analisa data, presentase nasabah yang tidak memiliki anak merupakan yang tertinggi melakukan gagal bayar, sedangkan presentase gagal bayar terendah adalah nasabah yang memiliki 3 atau lebih banyak anak. 
2. Terdapat hubungan antara status perkawinan dan probabilitas seseorang melakukan gagal bayar pinjaman.
Hipotesis ini dapat diterima, sebab status pernikahan cukup mempengaruhi probabilitas nasabah melakukan gagal bayar pinjaman. Pada data ditemukan presentase nasabah yang melakukan gagal bayar tertinggi adalah nasabah yang sudah menikah, sedangkan yang terendah adalah nasabah dengan status janda/duda.
3. Terdapat hubungan antara tingkat pendapatan dan probabilitas seseorang melakukan gagal bayar pinjaman.
Tingkat pendapatan nasabah sangat mempengaruhi probabilitas nasabah melakukan gagal bayar pinjaman, maka dari itu hipotesis ketiga ini dapat diterima. Nasabah dengan penghasilan besar memiliki hasil presentase gagal bayar yang sangat kecil dibanding dengan kelompok lainnya, dan sesuai hipotesis, nasabah dengan penghasilan yang tergolong rendah memiliki presentase gagal bayar tertinggi. 
4. Perbedaan tujuan pinjaman memengaruhi probabilitas seseorang melakukan gagal bayar pinjaman.
Perbedaan tujuan pinjaman juga memengaruhi probabilitas nasabah melakukan gagal bayar pinjaman, oleh karenanya, hipotesi terakhir juga dapat diterima. Nasabah yang bertujuan untuk pembelian real estate memiliki hasil presentase gagal bayar yang tertinggi dibanding lainnya, dan nasabah dengan tujuan biaya pernikahan memiliki presentase gagal bayar terendah. 
